In [146]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [147]:
imdb = pd.read_csv("IMDb Movies India.csv", encoding = "ISO-8859-1")
imdb

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali
...,...,...,...,...,...,...,...,...,...,...
15504,Zulm Ko Jala Doonga,(1988),NaN,Action,4.6,11,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand
15505,Zulmi,(1999),129 min,"Action, Drama",4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani
15506,Zulmi Raj,(2005),NaN,Action,NaN,NaN,Kiran Thej,Sangeeta Tiwari,NaN,NaN
15507,Zulmi Shikari,(1988),NaN,Action,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [149]:
#Look at the total number of unique values within the dataset
for column in imdb.columns:
    print(f'{column} has {imdb[column].nunique()} unique values')

Name has 13838 unique values
Year has 102 unique values
Duration has 182 unique values
Genre has 485 unique values
Rating has 84 unique values
Votes has 2034 unique values
Director has 5938 unique values
Actor 1 has 4718 unique values
Actor 2 has 4891 unique values
Actor 3 has 4820 unique values


In [150]:
#convert duration into numeric and then convert to seconds
imdb['Votes'] = imdb['Votes'].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x)
imdb['Votes'] = imdb['Votes'].apply(lambda x: float(x.replace('$', '').replace('M', '')) * 1e6 if isinstance(x, str) and 'M' in x else x)
imdb['Votes'] = pd.to_numeric(imdb['Votes'])
imdb['Votes'] = imdb['Votes'].astype('Int64')

In [151]:
#convert the data types for the neccessary columns for easier analysis
#for year remove the brackets first
imdb['Year'] = imdb['Year'].str.extract(r'(\d+)')
#Convert year to numeric
imdb['Year'] = pd.to_numeric(imdb['Year'], errors = 'coerce')
imdb['Year'] = imdb['Year'].astype('Int64') 


In [152]:
# Extract only the numeric part of the duration and convert it to integer
imdb['Duration'] = imdb['Duration'].str.extract(r'(\d+)').astype(float).astype('Int64')
#rename the column to identify it to be in minutes
imdb.rename(columns={'Duration': 'Duration(Min)'}, inplace=True)
#Create a new column for the duration in seconds
imdb['Duration(Sec)'] = imdb['Duration(Min)'] * 60
#display the dataframe to show that the changes have been made
imdb.head(10)

,Name,Year,Duration(Min),Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,Duration(Sec)
0,,<NA>,<NA>,Drama,NaN,<NA>,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia,<NA>
1,#Gadhvi (He thought he was Gandhi),2019,109,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid,6540
2,#Homecoming,2021,90,"Drama, Musical",NaN,<NA>,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana,5400
3,#Yaaram,2019,110,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor,6600
4,...And Once Again,2010,105,Drama,NaN,<NA>,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali,6300
5,...Aur Pyaar Ho Gaya,1997,147,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor,8820
6,...Yahaan,2005,142,"Drama, Romance, War",7.4,1086,Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma,8520
7,.in for Motion,2008,59,Documentary,NaN,<NA>,Anirban Datta,NaN,NaN,NaN,3540
8,?: A Question Mark,2012,82,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia,4920
9,@Andheri,2014,116,"Action, Crime, Thriller",4.0,11,Biju Bhaskar Nair,Augustine,Fathima Babu,Byon,6960


In [153]:
imdb['Genre'] = imdb['Genre'].str.split(', ')
imdb.head(10)

,Name,Year,Duration(Min),Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,Duration(Sec)
0,,<NA>,<NA>,[Drama],NaN,<NA>,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia,<NA>
1,#Gadhvi (He thought he was Gandhi),2019,109,[Drama],7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid,6540
2,#Homecoming,2021,90,"[Drama, Musical]",NaN,<NA>,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana,5400
3,#Yaaram,2019,110,"[Comedy, Romance]",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor,6600
4,...And Once Again,2010,105,[Drama],NaN,<NA>,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali,6300
5,...Aur Pyaar Ho Gaya,1997,147,"[Comedy, Drama, Musical]",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor,8820
6,...Yahaan,2005,142,"[Drama, Romance, War]",7.4,1086,Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma,8520
7,.in for Motion,2008,59,[Documentary],NaN,<NA>,Anirban Datta,NaN,NaN,NaN,3540
8,?: A Question Mark,2012,82,"[Horror, Mystery, Thriller]",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia,4920
9,@Andheri,2014,116,"[Action, Crime, Thriller]",4.0,11,Biju Bhaskar Nair,Augustine,Fathima Babu,Byon,6960


In [154]:
imdb = imdb.explode('Genre')
imdb

,Name,Year,Duration(Min),Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,Duration(Sec)
0,,<NA>,<NA>,Drama,NaN,<NA>,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia,<NA>
1,#Gadhvi (He thought he was Gandhi),2019,109,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid,6540
2,#Homecoming,2021,90,Drama,NaN,<NA>,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana,5400
2,#Homecoming,2021,90,Musical,NaN,<NA>,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana,5400
3,#Yaaram,2019,110,Comedy,4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor,6600
...,...,...,...,...,...,...,...,...,...,...,...
15505,Zulmi,1999,129,Drama,4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani,7740
15506,Zulmi Raj,2005,<NA>,Action,NaN,<NA>,Kiran Thej,Sangeeta Tiwari,NaN,NaN,<NA>
15507,Zulmi Shikari,1988,<NA>,Action,NaN,<NA>,NaN,NaN,NaN,NaN,<NA>
15508,Zulm-O-Sitam,1998,130,Action,6.2,20,K.C. Bokadia,Dharmendra,Jaya Prada,Arjun Sarja,7800
